# Creating command-line Tools

In [ ]:
from ctapipe.core import Tool, Component, TelescopeComponent
from ctapipe.core.traits import (
    Integer, Float, List, Dict, Unicode,
    TraitError, observe, FloatTelescopeParameter,
    Path,
)
import logging
from time import sleep
from astropy import units as u
from ctapipe.utils import get_dataset_path

In [ ]:
GAMMA_FILE = get_dataset_path('gamma_test.simtel.gz')

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [ ]:
class MyComponent(Component):
    """ A Component that does stuff """

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")


# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Path(
        help="input file name", 
        exists=None,  # set to True to require existing, False for requiring non-existing
        directory_ok=False,
    ).tag(config=True)
    outfile = Path(
        help="output file name",
        exists=False, directory_ok=False
    ).tag(config=True)

    @observe("outfile")
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class TelescopeWiseComponent(TelescopeComponent):
    """ a component that contains parameters that are per-telescope configurable """

    param = FloatTelescopeParameter(
        help="Something configurable with telescope patterns", default_value=5.0
    ).tag(config=True)

In [ ]:
MyComponent()

In [ ]:
AdvancedComponent()

`TelescopeComponents` need to have a subarray given to them in order to work (since they need one to turn a `TelescopeParameter` into a concrete list of values for each telescope.  Here we will give a dummy one:

In [ ]:
from ctapipe.instrument import SubarrayDescription, TelescopeDescription

subarray = SubarrayDescription(
    "Junk",
    tel_positions={1: (0.0, 0.0, 0.0) * u.m, 2: (1.0, 1.0, 0.0) * u.m},
    tel_descriptions={
        1: TelescopeDescription.from_name("LST", "LSTCam"),
        2: TelescopeDescription.from_name("MST", "NectarCam"),
    },
)
subarray.info()

In [ ]:
TelescopeWiseComponent(subarray=subarray)

This TelescopeParameters can then be set using a list of patterns like:
```python
component.param = [ 
    ("type", "LST*",3.0), 
    ("type", "MST*", 2.0), 
    (id, 25, 4.0) 
]
```

These get translated into per-telescope-id values once the subarray is registered.  After that one acccess the per-telescope id values via:
```python
component.param.tel[tel_id]
```

## Now create an executable Tool that contains the Components

In [ ]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = dict(
        infile='AdvancedComponent.infile',
        iterations='MyTool.iterations'
    )

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent, TelescopeWiseComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup_comp(self):
        # when constructing Components, you must add them to the 
        # list of registered instances using add_component. This allows
        # the full configuration to be tracked
        self.comp = self.add_component(MyComponent(parent=self))
        self.comp2 = self.add_component(SecondaryMyComponent(parent=self))

    def setup_advanced(self):
        self.advanced = self.add_component(AdvancedComponent(parent=self))
        self.comp3 = self.add_component(TelescopeWiseComponent(parent=self, subarray=subarray))


    def setup(self):
        self.setup_comp()
        self.setup_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [ ]:
tool=MyTool()

In [ ]:
tool.print_help()

The following  is equivalant to the user typing `mytool --help-all`

In [ ]:
tool.print_help(classes=True)

## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile gamma_test.simtel.gz --iterations=3
```

As Tools are intended to be exectutables, they are raising `SystemExit` on exit.
Here, we use them to demonstrate how it would work, so we catch the `SystemExit`.

In [ ]:
try:
    tool.run(argv=['--infile',  GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

In [ ]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 


try:
    tool.run(argv=['--log-level','INFO','--infile', GAMMA_FILE, '--iterations','3'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

here we change the log-level to DEBUG:

In [ ]:
try:
    tool.run(argv=['--log-level','DEBUG','--infile', GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [ ]:
tool.iterations = 1
tool.log_level = 0

try:
    tool.run(['--infile', GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

see what happens when a value is set that is not of the correct type:

In [ ]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [ ]:
tool.advanced.outfile = "Another.txt"

we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [ ]:
tool2 = MyTool()

In [ ]:
try:
    tool2.run(argv=['--config', 'Tools.json'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

In [ ]:
print(tool2.advanced.infile)

In [ ]:
print(tool2.config)

In [ ]:
tool2.is_setup

In [ ]:
tool3 = MyTool()

In [ ]:
tool3.is_setup

In [ ]:
tool3.initialize(argv=[])

In [ ]:
tool3.is_setup

In [ ]:
tool3

In [ ]:
tool

In [ ]:
tool.comp2

## Getting the configuration of an instance

In [ ]:
tool.get_current_config()

In [ ]:
tool.iterations = 12
tool.get_current_config()

## Writing a Sample Config File

In [ ]:
print(tool.generate_config_file())